<div>
<img src="https://www.nasa.gov/wp-content/uploads/2024/07/osdr-gl4hs-logo.png" width="600"/>
</div>

# **NOTEBOOK 4: Aligning sequencing reads to a reference genome**


In this notebook you will map (align) the sequence reads from the FASTQ files to the reference chromosome index (chr17) you built in the previous notebook.

## **Objectives of this notebook**
The primary objective of this notebook is to align the 2 FASTQ files to a reference genome index for chromosome 17. Once you perform the alignment, you will check the alignment using a command called `samtools`. You can learn more about the `samtools` command in this [Wikipedia article](https://en.wikipedia.org/wiki/SAMtools).

## **UNIX commands introduced in this notebook**


# Prepare your runtime environment for this lab

In [1]:
# mount the google drive
from google.colab import drive
drive.flush_and_unmount()
drive.mount("mnt")


Drive not mounted, so nothing to flush and unmount.
Mounted at mnt


In [2]:
# time the notebook
import datetime
start_time = datetime.datetime.now()
print('notebook start time: ', start_time.strftime('%Y-%m-%d %H:%M:%S'))

notebook start time:  2025-07-14 19:42:04


In [3]:
# import the os module which you'll use throughout the notebook
import os

In [4]:
# set FASTQ_DIR and verify it exists
FASTQ_DIR="/content/mnt/MyDrive/NASA/GL4HS/FASTQ"
if not os.path.exists(FASTQ_DIR):
  raise Exception("STOP! You have not finished the previous notebooks!")

In [5]:
# set REFERENCE_DIR and verify it exists
REFERENCE_DIR="/content/mnt/MyDrive/NASA/GL4HS/REFERENCE"
if not os.path.exists(REFERENCE_DIR):
  raise Exception("STOP! You have not finished the previous notebooks!")

In [6]:
# set STAR_DIR and verify it exists
STAR_DIR="/content/mnt/MyDrive/NASA/GL4HS/STAR"
if not os.path.exists(STAR_DIR):
  raise Exception("STOP! You have not finished the previous notebooks!")

In [7]:
# check for the compressed trimmed fastq files
if not os.path.exists(f"{FASTQ_DIR}/TRIM/PAIRED/reduced_r1_val_1.fq.gz") \
  or not os.path.exists(f"{FASTQ_DIR}/TRIM/PAIRED/reduced_r2_val_2.fq.gz"):
  raise Exception("STOP: one or both of the reduced trimmed fq.gz files do not exist. You must first run the previous notebooks!")

In [8]:
# install samtools
!sudo apt-get install samtools > /dev/null 2>&1

In [9]:
# check version of samtools
!samtools --version

samtools 1.13
Using htslib 1.13+ds
Copyright (C) 2021 Genome Research Ltd.

Samtools compilation details:
    Features:       build=configure curses=yes 
    CC:             gcc
    CPPFLAGS:       -frelease  -Wdate-time -D_FORTIFY_SOURCE=2
    CFLAGS:         -g -O2 -ffile-prefix-map=�BUILDPATH�=. -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werror=format-security
    LDFLAGS:        -Wl,-Bsymbolic-functions -flto=auto -Wl,-z,relro -Wl,-z,now
    HTSDIR:         
    LIBS:           
    CURSES_LIB:     -lcurses

HTSlib compilation details:
    Features:       build=configure plugins=yes, plugin-path=/usr/local/lib/htslib:/usr/local/libexec/htslib:: libcurl=yes S3=yes GCS=yes libdeflate=yes lzma=yes bzip2=yes htscodecs=1.1.1
    CC:             gcc
    CPPFLAGS:       -I. -DSAMTOOLS=1 -Wdate-time -D_FORTIFY_SOURCE=2
    CFLAGS:         -g -O2 -ffile-prefix-map=/build/htslib-TQtOKr/htslib-1.13+ds=. -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werro

# Align the reads to the reference

In [10]:
# unzip the reduce_trimmed fq.gz files as STAR only reads unzipped fastq
#
import os
if os.path.exists(f"{FASTQ_DIR}/TRIM/PAIRED/reduced_r1_val_1.fq.gz") and not os.path.exists(f"{FASTQ_DIR}/TRIM/PAIRED/reduced_r1_val_1.fq"):
  !gunzip -f {FASTQ_DIR}/TRIM/PAIRED/reduced_r1_val_1.fq.gz
if os.path.exists(f"{FASTQ_DIR}/TRIM/PAIRED/reduced_r2_val_2.fq.gz") and not os.path.exists(f"{FASTQ_DIR}/TRIM/PAIRED/reduced_r2_val_2.fq"):
  !gunzip -f {FASTQ_DIR}/TRIM/PAIRED/reduced_r2_val_2.fq.gz

## Use the `STAR` command to perform the alignment

Read Sections 3.1 and 3.2 of the [STAR manual](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf) to learn about the STAR command options for running mapping (alignment) jobs.

For more information, feel free to read [this tutorial](https://hbctraining.github.io/Intro-to-rnaseq-hpc-O2/lessons/03_alignment.html) on the STAR aligner for more information about how it works.

In [11]:
# run STAR to align the reads to the reference
# this may take a LONG time (several hours)
# runtime depends on the REDUCTION_FACTOR setting from the first notebook
# runtime also depends on how much data you're aligning
# runtime also depends on how much memory is on the computer where STAR is running
import datetime
start = datetime.datetime.now()
print('starting time: ', start.strftime('%Y-%m-%d %H:%M:%S'))

!chmod +x {STAR_DIR}/STAR-2.7.11b/bin/Linux_x86_64_static/STAR

if os.path.exists(f"{STAR_DIR}/ALIGNMENT"):
  !rm -rf {STAR_DIR}/ALIGNMENT
!mkdir {STAR_DIR}/ALIGNMENT
!{STAR_DIR}/STAR-2.7.11b/bin/Linux_x86_64_static/STAR \
  --outFileNamePrefix {STAR_DIR}/ALIGNMENT/chr17 \
  --readFilesIn {FASTQ_DIR}/TRIM/PAIRED/reduced_r1_val_1.fq {FASTQ_DIR}/TRIM/PAIRED/reduced_r2_val_2.fq \
  --genomeDir {REFERENCE_DIR}/MM39_CHR17 \
  --runThreadN 2

end = datetime.datetime.now()
print('ending time: ', end.strftime('%Y-%m-%d %H:%M:%S'))


starting time:  2025-07-14 19:42:26
	/content/mnt/MyDrive/NASA/GL4HS/STAR/STAR-2.7.11b/bin/Linux_x86_64_static/STAR --outFileNamePrefix /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17 --readFilesIn /content/mnt/MyDrive/NASA/GL4HS/FASTQ/TRIM/PAIRED/reduced_r1_val_1.fq /content/mnt/MyDrive/NASA/GL4HS/FASTQ/TRIM/PAIRED/reduced_r2_val_2.fq --genomeDir /content/mnt/MyDrive/NASA/GL4HS/REFERENCE/MM39_CHR17 --runThreadN 2
	STAR version: 2.7.11b   compiled: 2024-01-25T16:12:02-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Jul 14 19:42:28 ..... started STAR run
Jul 14 19:42:28 ..... loading genome
Jul 14 19:42:45 ..... started mapping
Jul 14 20:15:54 ..... finished mapping
Jul 14 20:15:54 ..... finished successfully
ending time:  2025-07-14 20:15:54


In [12]:
# check alignment output files
# there should be 5 files in all
# the alignment file itself is called chr17Aligned.out.sam
!ls -lh {STAR_DIR}/ALIGNMENT/chr17*

-rw------- 1 root root  25M Jul 14 20:15 /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17Aligned.out.sam
-rw------- 1 root root 2.0K Jul 14 20:15 /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17Log.final.out
-rw------- 1 root root 5.4K Jul 14 20:15 /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17Log.out
-rw------- 1 root root 1.8K Jul 14 20:15 /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17Log.progress.out
-rw------- 1 root root  24K Jul 14 20:15 /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17SJ.out.tab


## Use `samtools` view to examine the alignment

Read the [samtools view manpage](https://www.htslib.org/doc/samtools-view.html) to learn more about this command.

In [13]:
# look at the first 10 lines of the SAM file
# Question: Which position is the first read in this (unsorted) SAM file?
!samtools view -h {STAR_DIR}/ALIGNMENT/chr17Aligned.out.sam | head -10

@HD	VN:1.4
@SQ	SN:17	LN:95294699
@PG	ID:STAR	PN:STAR	VN:2.7.11b	CL:/content/mnt/MyDrive/NASA/GL4HS/STAR/STAR-2.7.11b/bin/Linux_x86_64_static/STAR   --runThreadN 2   --genomeDir /content/mnt/MyDrive/NASA/GL4HS/REFERENCE/MM39_CHR17   --readFilesIn /content/mnt/MyDrive/NASA/GL4HS/FASTQ/TRIM/PAIRED/reduced_r1_val_1.fq   /content/mnt/MyDrive/NASA/GL4HS/FASTQ/TRIM/PAIRED/reduced_r2_val_2.fq      --outFileNamePrefix /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17
@PG	ID:samtools	PN:samtools	PP:STAR	VN:1.13	CL:samtools view -h /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17Aligned.out.sam
@CO	user command line: /content/mnt/MyDrive/NASA/GL4HS/STAR/STAR-2.7.11b/bin/Linux_x86_64_static/STAR --outFileNamePrefix /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17 --readFilesIn /content/mnt/MyDrive/NASA/GL4HS/FASTQ/TRIM/PAIRED/reduced_r1_val_1.fq /content/mnt/MyDrive/NASA/GL4HS/FASTQ/TRIM/PAIRED/reduced_r2_val_2.fq --genomeDir /content/mnt/MyDrive/NASA/GL4HS/REFERENCE/MM39_CHR17 --runThr

In [14]:
# sort alignment and save in BAM file
!samtools sort {STAR_DIR}/ALIGNMENT/chr17Aligned.out.sam -o {STAR_DIR}/ALIGNMENT/chr17Aligned.out.bam

In [15]:
# look at the first 10 lines of the sorted BAM file
# Question: Which position is the first read in this (sorted) BAM file?
!samtools view -h {STAR_DIR}/ALIGNMENT/chr17Aligned.out.bam | head -10

@HD	VN:1.4	SO:coordinate
@SQ	SN:17	LN:95294699
@PG	ID:STAR	PN:STAR	VN:2.7.11b	CL:/content/mnt/MyDrive/NASA/GL4HS/STAR/STAR-2.7.11b/bin/Linux_x86_64_static/STAR   --runThreadN 2   --genomeDir /content/mnt/MyDrive/NASA/GL4HS/REFERENCE/MM39_CHR17   --readFilesIn /content/mnt/MyDrive/NASA/GL4HS/FASTQ/TRIM/PAIRED/reduced_r1_val_1.fq   /content/mnt/MyDrive/NASA/GL4HS/FASTQ/TRIM/PAIRED/reduced_r2_val_2.fq      --outFileNamePrefix /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17
@PG	ID:samtools	PN:samtools	PP:STAR	VN:1.13	CL:samtools sort -o /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17Aligned.out.bam /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17Aligned.out.sam
@PG	ID:samtools.1	PN:samtools	PP:samtools	VN:1.13	CL:samtools view -h /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17Aligned.out.bam
@CO	user command line: /content/mnt/MyDrive/NASA/GL4HS/STAR/STAR-2.7.11b/bin/Linux_x86_64_static/STAR --outFileNamePrefix /content/mnt/MyDrive/NASA/GL4HS/STAR/ALIGNMENT/chr17 --readF

In [16]:
# count all aligned reads in BAM file
!samtools view -c {STAR_DIR}/ALIGNMENT/chr17Aligned.out.bam

83004


In [17]:
# count number of reads with MAPQ quality score 20 or higher in BAM file
!samtools view -q 20 -c {STAR_DIR}/ALIGNMENT/chr17Aligned.out.bam

68828


In [18]:
# calculate percentage of reads with MAPQ > 20 in BAM file
q20 = !samtools view -q 20 -c {STAR_DIR}/ALIGNMENT/chr17Aligned.out.bam
total = !samtools view -c {STAR_DIR}/ALIGNMENT/chr17Aligned.out.bam
int(q20[0]) / int(total[0])

0.8292130499734952

## Use `samtools` flagstat to get statistics on the alignment

Read the [samtools flagstat manpage](https://www.htslib.org/doc/samtools-flagstat.html) for more information.

In [19]:
# get general stats on alignment from BAM file
!samtools flagstat {STAR_DIR}/ALIGNMENT/chr17Aligned.out.bam

83004 + 0 in total (QC-passed reads + QC-failed reads)
73918 + 0 primary
9086 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
83004 + 0 mapped (100.00% : N/A)
73918 + 0 primary mapped (100.00% : N/A)
73918 + 0 paired in sequencing
36962 + 0 read1
36956 + 0 read2
73906 + 0 properly paired (99.98% : N/A)
73906 + 0 with itself and mate mapped
12 + 0 singletons (0.02% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


Singletons are reads that are missing a mate (R1 or R2) in paired-end sequencing. You can read more [here](https://www.seqanswers.com/forum/bioinformatics/bioinformatics-aa/41311-what-is-singletons).

## Use the `samtools mpileup` command to examine individual base mappings

You can read more about samtools mpileup in the DESCRIPTION Pileup Format section of the [samtools manpage](https://www.htslib.org/doc/samtools-mpileup.html) and in this [Wikipedia article](https://en.wikipedia.org/wiki/Pileup_format).

In [20]:
# run mpileup in a region of the alignment in BAM file
!samtools mpileup -f {REFERENCE_DIR}/Mus_musculus.GRCm39.dna.chromosome.17.fa {STAR_DIR}/ALIGNMENT/chr17Aligned.out.bam | sed -n '1,30p'

[mpileup] 1 samples in 1 input files
17	3094414	A	1	^~.	;
17	3094415	T	1	.	;
17	3094416	C	1	T	;
17	3094417	G	1	.	F
17	3094418	G	1	.	F
17	3094419	G	1	.	J
17	3094420	A	1	.	J
17	3094421	C	1	.	J
17	3094422	C	1	.	J
17	3094423	T	1	.	J
17	3094424	G	1	.	J
17	3094425	G	1	.	J
17	3094426	A	1	.	J
17	3094427	T	1	.	J
17	3094428	C	2	.^~.	JA
17	3094429	T	2	..	JA
17	3094430	A	2	..	JF
17	3094431	T	2	..	JF
17	3094432	G	2	..	JF
17	3094433	C	2	..	JJ
17	3094434	T	2	..	JJ
17	3094435	T	2	..	JJ
17	3094436	C	2	..	JJ
17	3094437	T	2	..	JJ
17	3094438	T	2	..	JJ
17	3094439	G	2	..	JJ
17	3094440	C	2	..	JJ
17	3094441	G	1	.	J
17	3094442	C	2	..	FJ
17	3094443	C	2	..	JJ


# Check your work before moving on

In [21]:
# check disk space utilization (should be about 1.2GB)
!du -sh /content/mnt/MyDrive/NASA/GL4HS

1.7G	/content/mnt/MyDrive/NASA/GL4HS


In [22]:
# time the notebook
import datetime
end_time = datetime.datetime.now()
print('notebook end time: ', end_time.strftime('%Y-%m-%d %H:%M:%S'))

total_time = end_time - start_time
print('notebook total runtime: ', total_time)

notebook end time:  2025-07-14 20:16:15
notebook total runtime:  0:34:11.229464
